In [1]:
# Respons sistem 0.0
import ee

ee.Authenticate()
ee.Initialize()

In [2]:
# ----- User Input -----
# ----- Module 1 -----
# 1. AOI file path
UploadAOIVect = False
AOIVectPath = 'data/aoi_pedam.shp'
# AOIEePath = 'projects/ee-fazamahezs/assets/KHG_sample2'
AOIEePath = 'projects/ee-fazamahezs/assets/aoi_pedam'

# Define AOI
if UploadAOIVect:
    gdf = gpd.read_file(AOIVectPath)

    # Convert to EE FeatureCollection
    features = []
    for _, row in gdf.iterrows():
        geom = ee.Geometry(mapping(row.geometry))
        features.append(ee.Feature(geom, {}))
    AOI = ee.FeatureCollection(features).geometry()
else:
    AOI = ee.FeatureCollection(AOIEePath).geometry()

# 2. Landsat image
# ClippedImage = 

# Modul 2 (Define Classification Scheme)

In [4]:
# Import module and functions
import pandas as pd
from epistemx.module_2 import LULCSchemeClass, SaveClassificationScheme

In [22]:
# ----- Data Input -----
# 1. Choose default LULC scheme
ReferenceDataSource = False

# 1. Option to manual input or upload csv
ManualScheme = True # set as 'false' if you want to upload your own classes in csv file

# 2. Upload csv file if ManualScheme is 'false'
# LULCTablePath = "lc_tbl_KHG.csv"
LULCTablePath = "C:/Users/fmahezs/epistem/modules_notebook/data/lc_pedamaran.csv" # it can accept both string path and earth engine asset

In [23]:
# ----- System respons 2.1 -----
if ReferenceDataSource == True:
    # ----- System respons 2.1.c -----
    # LULCTable = ee.FeatureCollection() # call the restore+ classification scheme
    pass 
else:
    if ManualScheme == True:
        # ----- System respons 2.1.b (Manual/Interactive) -----
        LULCClassifier = LULCSchemeClass()
        LULCClassifier.Display()
    else:
        # ----- System respons 2.1.a (Load from Path/Asset) -----
        try:
            # Check input data source
            if isinstance(LULCTablePath, str) and (LULCTablePath.startswith('users/') or LULCTablePath.startswith('projects/') or LULCTablePath.startswith('ft:')):
                LULCTable = ee.FeatureCollection(LULCTablePath)
                print(f"Loaded Earth Engine FeatureCollection from Asset ID: {LULCTablePath}")
            elif isinstance(LULCTablePath, str):
                if 'sheet' in locals() and hasattr(sheet, 'to_dataframe'):
                     LULCTable = sheet.to_dataframe(LULCTablePath)
                else:
                     LULCTable = pd.read_csv(LULCTablePath)
                print(f"Loaded {len(LULCTable)} land cover/use classes from local CSV file: {LULCTablePath}")
            else:
                raise TypeError("LULCTablePath must be a string file path or an Earth Engine Asset ID.")
        except Exception as e:
            print(f"Error loading LULC Table from path/asset {LULCTablePath}: {e}")
            raise

In [24]:
# ----- System respons 2.2 -----
if ManualScheme == True:
    # Define variable
    LULCTable = LULCClassifier.GetDataframe()
    print("\nDefined Land cover/use classes:")
    display(LULCTable)
else:
    print("\nDefined Land cover/use classes:")
    display(LULCTable)


Defined Land cover/use classes:


,ID,Kelas Tutupan/Penggunaan Lahan,Color Palette
0,1,shrub,#2e8540
1,2,settlement,#382e85
2,3,waterbody,#6ce5c7


# Modul 3 (Define LULC Data Sample)

In [13]:
# Modul 3a 
# Import modules and functions
import ee
import pandas as pd
import geopandas as gpd
import ipysheet as sheet
from shapely.geometry import mapping
from epistemx.module_3a import InputCheck, SyncTrainData, SplitTrainData

In [17]:
# ----- Data Input -----
# 1. Decision to upload data
UploadTrainData = True

# 2. Training data file path (if UploadTrainData is true)
TrainVectPath  = 'C:/Users/fmahezs/epistem/modules_notebook/data/Pedamaran_sample.shp'
TrainEePath = 'projects/ee-fazamahezs/assets/pedamaran_points'
#TrainEePath = 'projects/ee-fazamahezs/assets/Pedamaran_sample'

# 3. Split data training
SplitTrain = False
TrainSplitPct = 0.7 

In [15]:
# ----- System response 3.1 -----
# Check if user chose default classification scheme in module 2
if ReferenceDataSource == True:
    InputCheck('ClippedImage', 'AOI')
    LULCTable = ee.FeatureCollection() # define restore+ LULC scheme
    TrainDataRaw = ee.FeatureCollection('projects/ee-rg2icraf/assets/Indonesia_lulc_Sample')
else:
    InputCheck('LULCTable', 'ClippedImage', 'AOI')

TypeError: InputCheck() takes no arguments

In [18]:
# ----- System response 3.2 -----
if UploadTrainData:

    # --- System response 3.2.a ---
    # Load training data
    TrainDataDict = SyncTrainData.LoadTrainData(
        landcover_df=LULCTable,
        aoi_geometry=AOI,
        training_shp_path=TrainVectPath if UploadTrainData else None,
        training_ee_path=None if UploadTrainData else TrainEePath
    )

    # Set class field
    TrainDataDict = SyncTrainData.SetClassField(TrainDataDict, 'LULC_Type')

    # Validate classes
    TrainDataDict = SyncTrainData.ValidClass(TrainDataDict)

    # Check sample sufficiency
    TrainDataDict = SyncTrainData.CheckSufficiency(TrainDataDict, min_samples=20)

    # Filter by AOI
    TrainDataDict = SyncTrainData.FilterTrainAoi(TrainDataDict)

    # Create training data table
    table_df, total_samples, insufficient_df = SyncTrainData.TrainDataRaw(
        training_data=TrainDataDict.get('training_data'),
        landcover_df=TrainDataDict.get('landcover_df'),
        class_field=TrainDataDict.get('class_field')
    )

    # --- Print summary ---
    vr = TrainDataDict.get('validation_results', {})

    print("=" * 70)
    print("TRAINING DATA SUMMARY")
    print("=" * 70)
    print(f"Total training points loaded     : {vr.get('total_points', 'N/A')}")
    print(f"Points after class filtering     : {vr.get('points_after_class_filter', 'N/A')}")
    print(f"Valid points (inside AOI)        : {vr.get('valid_points', 'N/A')}")
    print(f"Invalid classes found            : {len(vr.get('invalid_classes', []))}")
    print(f"Points outside AOI               : {len(vr.get('outside_aoi', []))}")
    print("=" * 70)

    # --- Display the main table ---
    if table_df is not None and not table_df.empty:
        display_df = table_df.copy()
        if 'Percentage' in display_df.columns:
            display_df['Percentage'] = display_df['Percentage'].apply(
                lambda x: f"{x:.2f}%" if isinstance(x, (int, float)) else x
            )
        display(display_df)
    else:
        print("No valid training data available to display.")


TRAINING DATA SUMMARY
Total training points loaded     : 115
Points after class filtering     : 115
Valid points (inside AOI)        : 97
Invalid classes found            : 0
Points outside AOI               : 18


,ID,LULC_class,Sample_Count,Percentage,Status
0,10,oilPalmMonoculture,2,2.06%,Insufficient
1,9,rubberMonoculture,11,11.34%,Insufficient
2,17,waterbody,17,17.53%,Insufficient
3,15,settlement,44,45.36%,Sufficient
4,13,shrub,12,12.37%,Insufficient
5,12,grassAndSavanna,11,11.34%,Insufficient


In [19]:
# ----- System response 3.2.b -----
if UploadTrainData == True:    

    if insufficient_df is not None:
        # Preview the insufficient classes
        print(f"\n{len(insufficient_df)} classes with insufficient samples:")
        insufficient_display = insufficient_df.copy()
        insufficient_display['Percentage'] = insufficient_display['Percentage'].apply(lambda x: f"{x:.2f}%")
        display(insufficient_display)

        # Add insufficient sample

        # Define table
        TrainDataRecap = TrainDataDict['training_data']
    else:
        print(f"\nAll classes have sufficient samples!")

        # Define table
        TrainDataRecap = TrainDataDict['training_data']


5 classes with insufficient samples:


,ID,LULC_class,Sample_Count,Percentage,Status
0,10,oilPalmMonoculture,2,2.06%,Insufficient
1,9,rubberMonoculture,11,11.34%,Insufficient
2,17,waterbody,17,17.53%,Insufficient
4,13,shrub,12,12.37%,Insufficient
5,12,grassAndSavanna,11,11.34%,Insufficient


In [ ]:
# ----- System response 3.3 -----
if UploadTrainData == False:

    # ----- System response 3.3.a -----
    

In [20]:
# ----- System response 3.4 -----
print(f"Recapitulation of selected training data sample:")
RecapTable = (TrainDataRecap.groupby(['kelas', 'LULC_Type'])
                  .size()
                  .reset_index(name='Samples')
                  .rename(columns={'kelas': 'ID'})
                  .sort_values('ID')
                  .reset_index(drop=True))

display(RecapTable)

Recapitulation of selected training data sample:


,ID,LULC_Type,Samples
0,9,rubberMonoculture,11
1,10,oilPalmMonoculture,2
2,12,grassAndSavanna,11
3,13,shrub,12
4,15,settlement,44
5,17,waterbody,17


In [21]:
# ----- System response 3.5 -----
# Split data training
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Split data process
if SplitTrain == True:

    # Split training data
    TrainDataFinal, ValidDataFinal = SplitTrainData.SplitProcess(
        TrainDataRecap, 
        TrainSplitPct=TrainSplitPct, 
        random_state=123
    )

    # Print split statistics
    print(f"Split data summary")
    print(f" - Training data: {len(TrainDataFinal)} samples")
    print(f" - Validation data: {len(ValidDataFinal)} samples")
    print(f" - Total: {len(TrainDataRecap)} samples\n")

    # Display preview
    print(f"Training data head:")
    display(TrainDataFinal.head())
    print(f"Validation data head:")
    display(ValidDataFinal.head())

    # Plot the splitted data
    Map = SplitTrainData.PlotTrainValidInteractive(TrainDataFinal, ValidDataFinal, AOI)
    Map

else:
    print("Split not performed. Using entire dataset as training.")
    TrainDataFinal = TrainDataRecap
    ValidDataFinal = None

    # Display preview
    print(f"Training data head:")
    display(TrainDataFinal.head())

    # Plot the training data
    Map = SplitTrainData.PlotTrainValidInteractive(TrainDataFinal, AOI)
    Map


Split not performed. Using entire dataset as training.
Training data head:


,kelas,LULC_Type,geometry
0,9,rubberMonoculture,POINT Z (104.84809 -3.51406 0)
1,9,rubberMonoculture,POINT Z (104.81615 -3.52955 0)
2,9,rubberMonoculture,POINT Z (104.83744 -3.516 0)
3,9,rubberMonoculture,POINT Z (104.89067 -3.54697 0)
4,9,rubberMonoculture,POINT Z (104.87809 -3.53148 0)
